In [1]:
import os
import qcfractal.interface as ptl
import offsb.qcarchive.qcatree as qca
import treedi.node as Node
import pandas as pd
import pickle
import qcelemental

In [3]:
# Initialize and configure which datasets to retreive
client = ptl.FractalClient()
with pd.option_context(
        'display.max_rows', 10000000, 
        'display.max_columns', 100,
        'display.max_colwidth', 10000):
    print(client.list_collections())

QCA = qca.QCATree( "QCA", root_payload=client, node_index=dict(), db=dict()) 

sets = [
    ( "TorsionDriveDataset", "OpenFF Gen 2 Torsion Set 2 Coverage" ),
    ( "TorsionDriveDataset", "OpenFF Gen 2 Torsion Set 2 Coverage 2" ),
    ( "OptimizationDataset", "OpenFF Gen 2 Opt Set 2 Coverage" )
]

for s in sets:
    ds = client.get_collection( *s)

    # Skip searching for Hessians on TD since they are (never) there?
    drop = ["Hessian"] if s[0] == "TorsionDriveDataset" else []

    # only get 2 objects per DS, just to test things
    QCA.build_index( ds, drop=drop, start=0, limit=2)

    # List specs and status
    specs = ds.list_specifications()
    for i in range(len(ds.list_specifications())):
        print(ds.status(specs.iloc[i]._name))


                                                                                                                                                                    tagline
collection              name                                                                                                                                               
Dataset                 ANI-1                                                                    22 million off-equilibrium conformations and enegies for organic molecules
                        COMP6 ANI-MD                                                                        Benchmark containing MD trajectories from the ANI-1x potential.
                        COMP6 DrugBank                                                                             Benchmark containing DrugBank off-equilibrium molecules.
                        COMP6 GDB10to13                                                Benchmark containing off-equilibrium molecules from G

Expanding spec default
Items found: 2
Chunk                   0    2     ... Received      2  | elapsed: 0:00:00.353246
TotalTime: 0:00:00.353496
Chunk                   0    2     ... Received      2  | elapsed: 0:00:00.328038
TotalTime: 0:00:00.328290
Chunk                   0  135     ... Received    135  | elapsed: 0:00:00.923113
TotalTime: 0:00:00.923411
Chunk with projection  1000 1651     ... Received    935  | elapsed: 0:00:00.829226... Received   1532  | elapsed: 0:00:00.642167
TotalTime: 0:00:01.473330
          default
COMPLETE       84
RUNNING         5
ERROR           4
Expanding spec default
Items found: 2
Chunk                   0    2     ... Received      2  | elapsed: 0:00:00.344235
TotalTime: 0:00:00.344620
Chunk                   0    2     ... Received      2  | elapsed: 0:00:00.329341
TotalTime: 0:00:00.329601
Chunk                   0  122     ... Received    122  | elapsed: 0:00:00.822387
TotalTime: 0:00:00.822660
Chunk with projection  1000 1499     ... Receive

In [4]:
# (Optional) Build the index out to include more than just 2 records

# The "last" child here is the Optimization dataset, so we are
# asking to build a full index of the optimization data
# skel=True signifies only metadata and lightweight info is retreived
# This will enable iteration, but the heavy data e.g. the geometries will
# not be available

if False:
    opts = QCA.root().children[-1]
    QCA.expand_qca_dataset_as_tree( opts, skel=True, drop=["Hessian"])

In [5]:
# Cache molecules locally by downloading final molecules in batches for speed

# This is a specialized iterator that will download all final molecules
# from each of the optimizations, in all of the datasets in the index.
# This means all torsion drive optimizations and regular optimizations
# will be collected.
QCA.cache_optimization_minimum_molecules()

Chunk                   0  259     ... Received    259  | elapsed: 0:00:01.892828
TotalTime: 0:00:01.893108


In [6]:
# Example of how to iterate over the index
# Iterate through the molecules, and combine data for the same molecule
# By default this is done by checking the 
# canonical_isomeric_explicit_hydrogen_mapped_smiles string in the entries.
# Creates a "folder" node, whose children are the entries which all
# have the same smiles, both torsiondrives and optimizations
for n in QCA.combine_by_entry():
    print(n.name, QCA.db[QCA[n.children[0]].payload]['entry'].attributes['canonical_smiles'])
    for e in n.children:
        record = QCA.db[QCA[e].payload]['data'] 
        if QCA[e].name == "TorsionDrive":
            print("    ", QCA[e].name, QCA[e].payload, record['status'][:], record['keywords'].dihedrals )
        else:
            print("    ", QCA[e].name, QCA[e].payload, record['status'][:])

Folder C(CBr)c1[nH]nnn1
     TorsionDrive QCP-18045415 COMPLETE [(0, 1, 2, 7)]
Folder C1C(O1)CCCF
     TorsionDrive QCP-18045416 COMPLETE [(1, 2, 3, 4)]
Folder C1C(O1)CCCF
     TorsionDrive QCP-18536017 COMPLETE [(2, 3, 4, 6)]
Folder C(C(F)(Cl)Br)(F)(F)Br
     TorsionDrive QCP-18536018 COMPLETE [(5, 1, 0, 6)]
Folder C1CC1NC(=O)CN2CC(CO2)O
     Optimization QCP-18433463 COMPLETE
     Optimization QCP-18433464 COMPLETE


In [7]:
# Iterate entries, showing nodes to root

for n in QCA.combine_by_entry():
    for e in n.children:
        e = QCA[e]
        for i,k in enumerate(QCA.node_iter_to_root(e)):
            print("  "*i,k)

 <Node Name=TorsionDrive Tree=QCA IDX=3 Payload=QCP-18045415>
   <Node Name=Specification Tree=QCA IDX=2 Payload=QCS-default>
     <Node Name=OpenFF Gen 2 Torsion Set 2 Coverage Tree=QCA IDX=1 Payload=246>
       <Node Name=ROOT Tree=None IDX=0 Payload=ROOT>
 <Node Name=TorsionDrive Tree=QCA IDX=4 Payload=QCP-18045416>
   <Node Name=Specification Tree=QCA IDX=2 Payload=QCS-default>
     <Node Name=OpenFF Gen 2 Torsion Set 2 Coverage Tree=QCA IDX=1 Payload=246>
       <Node Name=ROOT Tree=None IDX=0 Payload=ROOT>
 <Node Name=TorsionDrive Tree=QCA IDX=3492 Payload=QCP-18536017>
   <Node Name=Specification Tree=QCA IDX=3491 Payload=QCS-default>
     <Node Name=OpenFF Gen 2 Torsion Set 2 Coverage 2 Tree=QCA IDX=3490 Payload=257>
       <Node Name=ROOT Tree=None IDX=0 Payload=ROOT>
 <Node Name=TorsionDrive Tree=QCA IDX=3493 Payload=QCP-18536018>
   <Node Name=Specification Tree=QCA IDX=3491 Payload=QCS-default>
     <Node Name=OpenFF Gen 2 Torsion Set 2 Coverage 2 Tree=QCA IDX=3490 Payload=

In [8]:
# Iterate molecules, showing nodes to root
for n in QCA.node_iter_optimization_minimum(QCA.root()):
    for i,k in enumerate(QCA.node_iter_to_root(n)):
        print("  "*i,k)

 <Node Name=Molecule Tree=QCA IDX=1858 Payload=QCM-11767136>
   <Node Name=GradientStub Tree=QCA IDX=207 Payload=QCR-18047767>
     <Node Name=Optimization Tree=QCA IDX=6 Payload=QCP-18045586>
       <Node Name=Constraint Tree=QCA IDX=5 Payload=('dihedral', (0, 1, 2, 7), -60)>
         <Node Name=TorsionDrive Tree=QCA IDX=3 Payload=QCP-18045415>
           <Node Name=Specification Tree=QCA IDX=2 Payload=QCS-default>
             <Node Name=OpenFF Gen 2 Torsion Set 2 Coverage Tree=QCA IDX=1 Payload=246>
               <Node Name=ROOT Tree=None IDX=0 Payload=ROOT>
 <Node Name=Molecule Tree=QCA IDX=1872 Payload=QCM-11794461>
   <Node Name=GradientStub Tree=QCA IDX=221 Payload=QCR-18077419>
     <Node Name=Optimization Tree=QCA IDX=7 Payload=QCP-18067751>
       <Node Name=Constraint Tree=QCA IDX=5 Payload=('dihedral', (0, 1, 2, 7), -60)>
         <Node Name=TorsionDrive Tree=QCA IDX=3 Payload=QCP-18045415>
           <Node Name=Specification Tree=QCA IDX=2 Payload=QCS-default>
           

 <Node Name=Molecule Tree=QCA IDX=2737 Payload=QCM-2130955>
   <Node Name=GradientStub Tree=QCA IDX=1086 Payload=QCR-2773465>
     <Node Name=Optimization Tree=QCA IDX=94 Payload=QCP-2739422>
       <Node Name=Constraint Tree=QCA IDX=91 Payload=('dihedral', (1, 2, 3, 4), 60)>
         <Node Name=TorsionDrive Tree=QCA IDX=4 Payload=QCP-18045416>
           <Node Name=Specification Tree=QCA IDX=2 Payload=QCS-default>
             <Node Name=OpenFF Gen 2 Torsion Set 2 Coverage Tree=QCA IDX=1 Payload=246>
               <Node Name=ROOT Tree=None IDX=0 Payload=ROOT>
 <Node Name=Molecule Tree=QCA IDX=2748 Payload=QCM-2128198>
   <Node Name=GradientStub Tree=QCA IDX=1097 Payload=QCR-2770508>
     <Node Name=Optimization Tree=QCA IDX=95 Payload=QCP-2739423>
       <Node Name=Constraint Tree=QCA IDX=91 Payload=('dihedral', (1, 2, 3, 4), 60)>
         <Node Name=TorsionDrive Tree=QCA IDX=4 Payload=QCP-18045416>
           <Node Name=Specification Tree=QCA IDX=2 Payload=QCS-default>
             

       <Node Name=Constraint Tree=QCA IDX=3505 Payload=('dihedral', (2, 3, 4, 6), -75)>
         <Node Name=TorsionDrive Tree=QCA IDX=3492 Payload=QCP-18536017>
           <Node Name=Specification Tree=QCA IDX=3491 Payload=QCS-default>
             <Node Name=OpenFF Gen 2 Torsion Set 2 Coverage 2 Tree=QCA IDX=3490 Payload=257>
               <Node Name=ROOT Tree=None IDX=0 Payload=ROOT>
 <Node Name=Molecule Tree=QCA IDX=5273 Payload=QCM-12262421>
   <Node Name=GradientStub Tree=QCA IDX=3774 Payload=QCR-18588131>
     <Node Name=Optimization Tree=QCA IDX=3507 Payload=QCP-18582014>
       <Node Name=Constraint Tree=QCA IDX=3505 Payload=('dihedral', (2, 3, 4, 6), -75)>
         <Node Name=TorsionDrive Tree=QCA IDX=3492 Payload=QCP-18536017>
           <Node Name=Specification Tree=QCA IDX=3491 Payload=QCS-default>
             <Node Name=OpenFF Gen 2 Torsion Set 2 Coverage 2 Tree=QCA IDX=3490 Payload=257>
               <Node Name=ROOT Tree=None IDX=0 Payload=ROOT>
 <Node Name=Molecule Tr

       <Node Name=Constraint Tree=QCA IDX=3589 Payload=('dihedral', (5, 1, 0, 6), -60)>
         <Node Name=TorsionDrive Tree=QCA IDX=3493 Payload=QCP-18536018>
           <Node Name=Specification Tree=QCA IDX=3491 Payload=QCS-default>
             <Node Name=OpenFF Gen 2 Torsion Set 2 Coverage 2 Tree=QCA IDX=3490 Payload=257>
               <Node Name=ROOT Tree=None IDX=0 Payload=ROOT>
 <Node Name=Molecule Tree=QCA IDX=5919 Payload=QCM-12242524>
   <Node Name=GradientStub Tree=QCA IDX=4420 Payload=QCR-18567018>
     <Node Name=Optimization Tree=QCA IDX=3591 Payload=QCP-18564425>
       <Node Name=Constraint Tree=QCA IDX=3589 Payload=('dihedral', (5, 1, 0, 6), -60)>
         <Node Name=TorsionDrive Tree=QCA IDX=3493 Payload=QCP-18536018>
           <Node Name=Specification Tree=QCA IDX=3491 Payload=QCS-default>
             <Node Name=OpenFF Gen 2 Torsion Set 2 Coverage 2 Tree=QCA IDX=3490 Payload=257>
               <Node Name=ROOT Tree=None IDX=0 Payload=ROOT>
 <Node Name=Molecule Tr

In [9]:
# Custom general iteration, where we iterate over molecules collated by:
# Molecule identity
#    Dihedral scanned
#        Final molecule ID and energy
for folder in QCA.combine_by_entry():
    print(folder)
    for entry in QCA.node_iter_entry(folder):
        print("    ", entry)
        for constraint in QCA.node_iter_depth_first(entry, select="Constraint"):
            print("      ", constraint)
            for molecule in QCA.node_iter_optimization_minimum(constraint):
                print("        ", molecule)

                # to give an idea, this will convert to a full QCArchive type
                qc_mol = qcelemental.models.molecule.Molecule.from_data(
                    QCA.db[molecule.payload]["data"])

<Node Name=Folder Tree=None IDX= Payload=<function match_canonical_isomeric_explicit_hydrogen_smiles at 0x7f1f4c9a6ae8>>
     <Node Name=TorsionDrive Tree=QCA IDX=3 Payload=QCP-18045415>
       <Node Name=Constraint Tree=QCA IDX=5 Payload=('dihedral', (0, 1, 2, 7), -60)>
         <Node Name=Molecule Tree=QCA IDX=1858 Payload=QCM-11767136>
         <Node Name=Molecule Tree=QCA IDX=1872 Payload=QCM-11794461>
         <Node Name=Molecule Tree=QCA IDX=1887 Payload=QCM-11793593>
         <Node Name=Molecule Tree=QCA IDX=1896 Payload=QCM-11816597>
       <Node Name=Constraint Tree=QCA IDX=10 Payload=('dihedral', (0, 1, 2, 7), 180)>
         <Node Name=Molecule Tree=QCA IDX=1923 Payload=QCM-11767403>
         <Node Name=Molecule Tree=QCA IDX=1931 Payload=QCM-11793940>
         <Node Name=Molecule Tree=QCA IDX=1939 Payload=QCM-11794177>
       <Node Name=Constraint Tree=QCA IDX=14 Payload=('dihedral', (0, 1, 2, 7), -165)>
         <Node Name=Molecule Tree=QCA IDX=1948 Payload=QCM-11781845>
   

         <Node Name=Molecule Tree=QCA IDX=5492 Payload=QCM-12289557>
         <Node Name=Molecule Tree=QCA IDX=5504 Payload=QCM-12336251>
       <Node Name=Constraint Tree=QCA IDX=3540 Payload=('dihedral', (2, 3, 4, 6), -15)>
         <Node Name=Molecule Tree=QCA IDX=5517 Payload=QCM-12262570>
         <Node Name=Molecule Tree=QCA IDX=5527 Payload=QCM-12291261>
         <Node Name=Molecule Tree=QCA IDX=5537 Payload=QCM-12311790>
       <Node Name=Constraint Tree=QCA IDX=3544 Payload=('dihedral', (2, 3, 4, 6), 15)>
         <Node Name=Molecule Tree=QCA IDX=5547 Payload=QCM-12262532>
         <Node Name=Molecule Tree=QCA IDX=5557 Payload=QCM-12287809>
       <Node Name=Constraint Tree=QCA IDX=3547 Payload=('dihedral', (2, 3, 4, 6), 105)>
         <Node Name=Molecule Tree=QCA IDX=5567 Payload=QCM-12257515>
         <Node Name=Molecule Tree=QCA IDX=5578 Payload=QCM-12288295>
       <Node Name=Constraint Tree=QCA IDX=3550 Payload=('dihedral', (2, 3, 4, 6), -120)>
         <Node Name=Molecul

In [10]:
# Utility functions to save data locally and run analysis, such as OpenMM energy

# Some helper functions to run OpenMM energy calculations
# The default here is to calculate single points for each of the
# downloaded molecules, from QCA.cache_optimization_minimum_molecules
def run_openMM( QCA, ff, ffname, targets=None):
    from offsb.op import openmm
    oMM = openmm.OpenMMEnergy( ff, QCA, ffname)
    oMM.apply( targets=targets)
    save( oMM)
    return oMM

def save( tree):
    name = os.path.join(".", tree.name + ".p")
    print("Saving: ", tree.ID, "as", name, end=" ... ")
    tree.to_pickle( db=True, name=name)
    print( "{:12.1f} MB".format(os.path.getsize( name)/1024**2))

In [11]:
# Save the index and db to disk
# Save the index and db. This can be saved and loaded to disk without issue.
# Useful once all molecules have been downloaded, and all future processing
# can happen locally without having to reconnect to the remote server
save(QCA)

Saving:  TREE-0.QCA as ./QCA.p ...          5.1 MB


In [12]:
# Example of how to load the data from disk
# Default is to save using the "name' as the filename (see constructors)
if False:
    with open("QCA.p", 'rb') as fid:
        QCA = pickle.load(fid)
    with open("oMM.oFF-Parsley-uncons.p", 'rb') as fid:
        oMM = pickle.load(fid)

In [13]:
# Example of how to get a list of all the entries
ds_nodes = [ QCA[index] for index in QCA.root().children]
entries = list(QCA.iter_entry( ds_nodes))

In [14]:
# Run the openMM calculation.
# Optionally takes a targets arg, which will iterate over a select number
# of entries. Default is to iterate over all entries (in all datasets) in the
# index.
run_openMM( QCA, 
    'openff_unconstrained-1.0.0.offxml', 
    'oMM.oFF-Parsley-uncons')

RDKit WARNING: [02:30:32] Enabling RDKit 2019.09.1 jupyter extensions


Building PartitionTree oMM.oFF-Parsley-uncons
Tree database using type <class 'dict'>
Searching .
Searching /home/tgokey/.cache/Python-Eggs/openforcefields-1.0.0-py3.7.egg-tmp/openforcefields/offxml/openff_unconstrained-1.0.0.offxml
Found


Warning (not error because allow_undefined_stereo=True): 


My db is {}


Warning (not error because allow_undefined_stereo=True): 


...

3 / 6 <Node Name=TorsionDrive Tree=QCA IDX=3492 Payload=QCP-18536017>
min mol is <Node Name=Molecule Tree=QCA IDX=5235 Payload=QCM-12229967> ene is -371.0485080833861 au
    <Node Name=Molecule Tree=QCA IDX=5819 Payload=QCM-12304237> [('dihedral', (2, 3, 4, 6), -165)] 116.5769181384141 kcal/mol
    <Node Name=Molecule Tree=QCA IDX=5830 Payload=QCM-12323115> [('dihedral', (2, 3, 4, 6), -165)] 116.05073164680554 kcal/mol
    <Node Name=Molecule Tree=QCA IDX=5838 Payload=QCM-12331249> [('dihedral', (2, 3, 4, 6), -165)] 116.06325755733064 kcal/mol
    <Node Name=Molecule Tree=QCA IDX=5759 Payload=QCM-12295286> [('dihedral', (2, 3, 4, 6), -150)] 117.48687603785908 kcal/mol
    <Node Name=Molecule Tree=QCA IDX=5772 Payload=QCM-12313121> [('dihedral', (2, 3, 4, 6), -150)] 117.48848930869073 kcal/mol
    <Node Name=Molecule Tree=QCA IDX=5780 Payload=QCM-12327337> [('dihedral', (2, 3, 4, 6), -150)] 117.13399633584632 kcal/mol
    <Node Name=Molecule Tree=QCA IDX=5789 Payload=QCM-12332870> [('di

Warning (not error because allow_undefined_stereo=True): 


4 / 6 <Node Name=TorsionDrive Tree=QCA IDX=3493 Payload=QCP-18536018>
min mol is <Node Name=Molecule Tree=QCA IDX=5945 Payload=QCM-12231342> ene is -5983.351017222989 au
    <Node Name=Molecule Tree=QCA IDX=6034 Payload=QCM-12239770> [('dihedral', (5, 1, 0, 6), -165)] 19.848926991856313 kcal/mol
    <Node Name=Molecule Tree=QCA IDX=6052 Payload=QCM-12249818> [('dihedral', (5, 1, 0, 6), -165)] 19.88478324831293 kcal/mol
    <Node Name=Molecule Tree=QCA IDX=6280 Payload=QCM-12242065> [('dihedral', (5, 1, 0, 6), -150)] 23.321803391630592 kcal/mol
    <Node Name=Molecule Tree=QCA IDX=6302 Payload=QCM-12273351> [('dihedral', (5, 1, 0, 6), -150)] 23.30019144516941 kcal/mol
    <Node Name=Molecule Tree=QCA IDX=6443 Payload=QCM-12249514> [('dihedral', (5, 1, 0, 6), -135)] 26.70085789732979 kcal/mol
    <Node Name=Molecule Tree=QCA IDX=6452 Payload=QCM-12308573> [('dihedral', (5, 1, 0, 6), -135)] 26.67668749913102 kcal/mol
    <Node Name=Molecule Tree=QCA IDX=6587 Payload=QCM-12273515> [('dihed

Warning (not error because allow_undefined_stereo=True): 


5 / 6 <Node Name=Optimization Tree=QCA IDX=6664 Payload=QCP-18433463>
min mol is <Node Name=Molecule Tree=QCA IDX=6830 Payload=QCM-12123759> ene is -648.4883365021865 au
    <Node Name=Molecule Tree=QCA IDX=6830 Payload=QCM-12123759> [] 154.59453168698684 kcal/mol
6 / 6 <Node Name=Optimization Tree=QCA IDX=6665 Payload=QCP-18433464>
min mol is <Node Name=Molecule Tree=QCA IDX=6875 Payload=QCM-12118925> ene is -648.4883976121773 au
    <Node Name=Molecule Tree=QCA IDX=6875 Payload=QCM-12118925> [] 154.65931944487954 kcal/mol
Saving:  TREE-1.oMM.oFF-Parsley-uncons as ./oMM.oFF-Parsley-uncons.p ...          0.3 MB


In [15]:
# See the other examples of how to use this analysis and plot the results. Start with 
# combine_QCA_and_analysis.py then plot_torsiondrive.py